# ASPLOS '23 LeaFTL Experiment

This is a simple experiment that run benchmarks with multi workload and mechanism in Chameleon Cloud.

## Create experiment container

This container provides the following:

- One node of any types ([see all types](https://chameleoncloud.readthedocs.io/en/latest/technical/reservations.html#chameleon-node-types))
- One public IP

### Configuration

Enter your project ID in the code block below.

In [ ]:
import chi

chi.use_site("CHI@UC")
chi.set("project_name", "CH-821904")

print(f'Using Project {chi.get("project_name")}')

In [ ]:
import os

USER = os.getenv('USER')
print(USER)

### Create reservation

Chameleon resources need to be reserved before they can be used. 
We will reserve one bare metal node and one public IP address, for right now.

If you get an error such as "no host available", it may be the case that all of our nodes are reserved. Check the availiablility calendar to see if this is true:
https://chi.uc.chameleoncloud.org/project/leases/calendar/host/

It may take around a minute or so for your lease to become active.

In [ ]:
import time
import keystoneauth1, blazarclient
from chi import lease

reservations = []
LEASE_KEY = f"{USER}-asplos23-leaftl"
lease_node_type = "compute_cascadelake_r"

try:
    lease_obj=lease.get_lease(LEASE_KEY)
except Exception as e:
    lease_obj=None

if lease_obj!=None:
    print("lease already created")
    print(lease_obj.node_reservations[0]["id"])
    # print(lease_obj.node_reservations["id"])
else:
    try:
        print(f"Creating lease with name = {LEASE_KEY}...")
        lease.add_fip_reservation(reservations, count=1)
        lease.add_node_reservation(reservations, count=1, node_type=lease_node_type)

        start_date, end_date = lease.lease_duration(hours=2, days=0)

        l = lease.create_lease(
            LEASE_KEY, 
            reservations, 
            start_date=start_date, 
            end_date=end_date
        )
        # l = lease.create_lease(
        #     LEASE_KEY, 
        #     reservations
        # )
        # print(l)
        lease_id = l["id"]
        lease_obj=lease.get_lease(LEASE_KEY)

        print("Waiting for lease to start ...")
        lease_obj.wait()
        print("Lease is now active!")
    #except keystoneauth1.exceptions.http.Unauthorized as e:
        #print("Unauthorized.\nDid set your project name and and run the code in the first cell?")
    except blazarclient.exception.BlazarClientException as e:
        print(f"There is an issue making the reservation. Please check the host calendar.")
        print("https://chi.uc.chameleoncloud.org/project/leases/calendar/host/")
        print(e)
    except Exception as e:
        print("An unexpected error happened.")
        print(e)

### Provision bare metal node

Next, we will launch the reserved node with an image. 
It will take approximately 10 minutes for the bare metal node to be successfully provisioned. 

This step takes the longest. First, our controller node must configure the requested node, which first sets up a deploy image. This image then downloads and copies the real image onto the hard drive, and the node is configured to reboot to the new OS. 

You can browse the images we offer in our appliance catalog: http://chameleoncloud.org/appliances

In [ ]:
from chi import server

image = "CC-Ubuntu20.04"
# print(l["reservations"][0]["id"])
try:
    s=server.get_server(LEASE_KEY)
except Exception as e:
    s=None

if s!=None:
    print("server already started")
else:
    try:
        s = server.create_server(
            LEASE_KEY, 
            image_name=image,
            reservation_id=lease_obj.node_reservations[0]["id"]
        )
    except Exception as e:
        print("An unexpected error happened.")
        print(e)

    print("Waiting for server to start ...")
    server.wait_for_active(s.id)
    print("Done")

By default our node is only connected to a private network and thus not reachable over the internet or via Jupyter here. We need to associate a "Floating IP" to the node, which gives it the public address we reserved.

In [ ]:
from chi import lease

floating_ip = lease_obj.get_reserved_floating_ips()[0]

try:
    ip=s.get_floating_ip()
except Exception as e:
    ip=None

if ip != None:
    print("server already get the ip")
else:
    server.associate_floating_ip(s.id, floating_ip_address=floating_ip)

Try to connect to the server through SSH

In [ ]:
print(f"Waiting for SSH connectivity on {floating_ip} ...")
timeout = 60 * 2
import socket
import time
# Repeatedly try to connect via SSH.
start_time = time.perf_counter()
while True:
    try:
        with socket.create_connection((floating_ip, 22), timeout=timeout):
            print("Connection successful")
            break
    except OSError as ex:
        time.sleep(10)
        if time.perf_counter() - start_time >= timeout:
            print(f"Timeout: after {timeout} seconds, could not connect via SSH. please wait until SSH is up and ready")

In [ ]:
## uncomment if your runtime is disconected but, the compile is not finished

# from chi import ssh
# import scripts.session as session

# session_data = session.load()
# floating_ip = session_data["floating_ip"]
# print(f"Waiting for SSH connectivity on {floating_ip} ...")
# timeout = 60 * 2
# import socket
# import time
# # Repeatedly try to connect via SSH.
# start_time = time.perf_counter()
# while True:
#     try:
#         with socket.create_connection((floating_ip, 22), timeout=timeout):
#             print("Connection successful")
#             break
#     except OSError as ex:
#         time.sleep(10)
#         if time.perf_counter() - start_time >= timeout:
#             print(f"Timeout: after {timeout} seconds, could not connect via SSH. please wait until SSH is up and ready")    

Test whether we can run commands through SSH

In [ ]:
from chi import ssh

with ssh.Remote(floating_ip) as conn:
    
    # conn.run("lscpu")
    conn.run("uname -a")

Setup the environment and install all necessary packages

In [ ]:
with ssh.Remote(floating_ip) as conn:
    conn.put("setup.sh")
    conn.run("bash ~/setup.sh")

Run experiment

In [ ]:
with ssh.Remote(floating_ip) as conn:
    conn.put("run_exp.sh")
    conn.run("bash ~/run_exp.sh")

Get the graph back to local

In [ ]:
with ssh.Remote(floating_ip) as conn:
    conn.get("LeaFTL/wiscsee/leaftl_scripts/plots/memory.png")